# RDDs + Programación Funcional

# Creamos un contexo para crear RDDs

In [1]:
import pandas as pd
!pip install pyspark --quiet
from pyspark import SparkContext
sc = SparkContext(master = "local", appName = "Transformaciones sobre un RDD")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


# Cargamos un RDDs

Para la realzación del ejercicio, con ayuda del archivo paises, realizaremos el equivalente de operaciones 'select','count','group by' y 'filter / where'

Cambia el valor de la ruta para que apunte a la ruta donde tienes los datos

In [3]:
from google.colab import drive
drive.mount('/content/drive/')
#LEER CSV
equiposOlimpicosRDD = sc.textFile('/content/drive/MyDrive/paises.csv').map(lambda line : line.split(","))

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
equiposOlimpicosRDD.take(10)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG'],
 ['5', 'Afghanistan', 'AFG'],
 ['6', 'Akatonbo', 'IRL'],
 ['7', 'Alain IV', 'SUI'],
 ['8', 'Albania', 'ALB'],
 ['9', 'Alcaid', 'POR']]

### Deshacemos un RDD

Con ayuda del método `collect()`, permite desparalelizar un RDD.


In [ ]:
equiposOlimpicosRDD.collect()

Creamos un dataframe a partir de un RDD mediante `pd.DataFrame()`

In [35]:
pd.DataFrame(equiposOlimpicosRDD.collect()[1:], columns = equiposOlimpicosRDD.collect()[0])

,id,equipo,sigla
0,1,30. Februar,AUT
1,2,A North American Team,MEX
2,3,Acipactli,MEX
3,4,Acturus,ARG
4,5,Afghanistan,AFG
...,...,...,...
1179,1180,Yugoslavia-2,YUG
1180,1181,Zambia,ZAM
1181,1182,Zefyros,GRE
1182,1183,Zimbabwe,ZIM


# Operaciones sobre un RDDs

Para la realzación del ejercicio, con ayuda del archivo paises, realizaremos el equivalente de operaciones `select`,`count`,`group by` , `filter` y `where`

### Ejercicio 1:
**Mostrar los países sin repetidos**

*   Seleccionar la columna de las siglas PISTA: Usa un map y quedate con la columna con indice 2.
*   Usamos el `distinct()`



In [14]:
#Inserta aquí tu código
columna_siglas = equiposOlimpicosRDD.map(lambda fila: fila[2])
columna_siglas_sin_repetidos = columna_siglas.distinct()
columna_siglas_sin_repetidos.take(5)

['sigla', 'AUT', 'MEX', 'ARG', 'AFG']

### Ejercicio 2
**Contar la cantidad de siglas diferentes de los equipos olímpicos existentes**

PISTA: Puedes usar `count()`

In [18]:
#Inserta aquí tu código
columna_siglas_sin_repetidos_len = columna_siglas_sin_repetidos.count()
columna_siglas_sin_repetidos_len

231

### Ejercicio 3 (Díficil, puedes hacerlo el último)
**Agrupamos datos para poder determinar cuántos equipos posee un país**

Recuerda que los RDDs poseen una estructura de `clave-valor`, por lo cual debemos poner primero el valor 'clave' (la sigla del país).

Con 'mapValues', al componente 'valor' le indicamos qué operación deseamos que se le aplique.

**Objetivo 1**: Tener un diccionario con `clave` la sigla del país y `valor` la lista de equipos de ese país

Con `groupByKey` podemos agrupar por `clave`, en este caso por sigla del país  

In [ ]:
#Inserta aquí tu código


**Objetivo 2**: Tener un diccionario con `clave` la sigla del país y `valor` la cantidad de equipos de ese país

In [ ]:
#Inserta aquí tu código


### Operación filter para obtener un subconjunto

Con el método 'filter', reducimos el conjuntos de equipos.

Nos quedamos con la sigla equivalente de argentina

In [19]:
equiposArgentinos = equiposOlimpicosRDD.filter(lambda l : "ARG" in l)
equiposArgentinos.collect()

[['4', 'Acturus', 'ARG'],
 ['37', 'Antares', 'ARG'],
 ['42', 'Arcturus', 'ARG'],
 ['43', 'Ardilla', 'ARG'],
 ['45', 'Argentina', 'ARG'],
 ['46', 'Argentina-1', 'ARG'],
 ['47', 'Argentina-2', 'ARG'],
 ['119', 'Blue Red', 'ARG'],
 ['238', 'Covunco III', 'ARG'],
 ['252', 'Cupidon III', 'ARG'],
 ['288', 'Djinn', 'ARG'],
 ['436', 'Gullvinge', 'ARG'],
 ['644', 'Matrero II', 'ARG'],
 ['672', 'Mizar', 'ARG'],
 ['774', 'Pampero', 'ARG'],
 ['843', 'Rampage', 'ARG'],
 ['1031', 'Tango', 'ARG'],
 ['1162', 'Wiking', 'ARG']]

# Accciones sobre RDDs

In [20]:
deportistaOlimpicoRDD = sc.textFile('/content/drive/MyDrive/deportista.csv').map(lambda line : line.split(","))
deportistaOlimpico2RDD = sc.textFile('/content/drive/MyDrive/deportista2.csv').map(lambda line : line.split(","))

In [21]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.union(deportistaOlimpico2RDD)

### Formas de visualizar datos de un RDDs

La operación 'take' nos devuelve 'N' valores que encuentre spark.

La operación 'top', previo ordena respecto al valor llave y nos devuelve 'N' valores.

La operacion 'takeSample', nos devuelve una muestra aleatoria de los valores, Observa que recibe tres parametros

| Orden | Argumento | Descripción | Valor
|-------|--------|-----|--------|
|1|withReplacement|Indica si la muestra podrá traer replicados|Bool|
|2| num| Cantidad de valores a retornar|int|
|3|seed| Semilla para el generador aleatorio|int|

Nota: Si encuentras complicado leer el código en los segmentos donde usamos indices en las listas, ejecutalo por partes para que visualizes que componentes seleccionamos

Nota: Para hacer join con RDDs, debemos tener selecionada la llave al inicio del RDD para poder realizar el cruce.

In [30]:
deportistaOlimpicoRDD.take(3)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199']]

In [31]:
deportistaOlimpicoRDD.map(lambda x :[x[-1], x[:-1]]).take(3)

[['equipo_id',
  ['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso']],
 ['199', ['1', 'A Dijiang', '1', '24', '180', '80']],
 ['199', ['2', 'A Lamusi', '1', '23', '170', '60']]]

In [32]:
equiposOlimpicosRDD.map(lambda x : [x[0],x[2]]).take(3)

[['id', 'sigla'], ['1', 'AUT'], ['2', 'MEX']]

In [36]:
deportistaOlimpicoRDD.map(lambda x :[x[-1], x[:-1]]).join(
    equiposOlimpicosRDD.map(lambda x : [x[0],x[2]])
    ).take(6)

[('199', (['1', 'A Dijiang', '1', '24', '180', '80'], 'CHN')),
 ('199', (['2', 'A Lamusi', '1', '23', '170', '60'], 'CHN')),
 ('199', (['602', 'Abudoureheman', '1', '22', '182', '75'], 'CHN')),
 ('199', (['1463', 'Ai Linuer', '1', '25', '160', '62'], 'CHN')),
 ('199', (['1464', 'Ai Yanhan', '2', '14', '168', '54'], 'CHN')),
 ('199', (['3605', 'An Weijiang', '1', '22', '178', '72'], 'CHN'))]

In [40]:
deportistaOlimpicoRDD.map(lambda x :[x[-1], x[:-1]]).join(
    equiposOlimpicosRDD.map(lambda x : [x[0],x[2]])
    ).top(5)

[('999', (['92679', 'Trygve Bjarne Pedersen', '1', '35', '0', '0'], 'NOR')),
 ('999', (['1144', 'Henrik Agersborg', '1', '47', '0', '0'], 'NOR')),
 ('999', (['10765', 'Einar Berntsen', '1', '28', '0', '0'], 'NOR')),
 ('998',
  (['111659', 'G. Bernard Bernie Skinner', '1', '34', '182', '82'], 'CAN')),
 ('996', (['116030', 'Edward Eddy Stutterheim', '1', '39', '0', '0'], 'NED'))]

In [45]:
deportistaOlimpicoRDD.map(lambda x :[x[-1], x[:-1]]).join(
    equiposOlimpicosRDD.map(lambda x : [x[0],x[2]])
    ).takeSample(False,4,10)

[('93',
  (['76502', 'Yelena Viktorovna Matoshko', '2', '30', '177', '80'], 'BLR')),
 ('487', (['110997', 'Ajit Singh', '1', '23', '185', '73'], 'IND')),
 ('249',
  (['41654', 'Toms Pedro Gonzlez Barrios', '1', '21', '178', '75'], 'CUB')),
 ('259', (['7157', 'Jlius Bal', '1', '22', '0', '0'], 'TCH'))]

### Ejercicio 4
**Muestra 17 filas del RDD deportistaOlimpicoRDD**


In [49]:
#Inserta aquí tu código
deportistaOlimpicoRDD.take(18)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278'],
 ['5', 'Christine Jacoba Aaftink', '2', '21', '185', '82', '705'],
 ['6', 'Per Knut Aaland', '1', '31', '188', '75', '1096'],
 ['7', 'John Aalberg', '1', '31', '183', '72', '1096'],
 ['8', 'Cornelia Cor Aalten Strannood ', '2', '18', '168', '0', '705'],
 ['9', 'Antti Sami Aalto', '1', '26', '186', '96', '350'],
 ['10', 'Einar Ferdinand Einari Aalto', '1', '26', '0', '0', '350'],
 ['11', 'Jorma Ilmari Aalto', '1', '22', '182', '76.5', '350'],
 ['12', 'Jyri Tapani Aalto', '1', '31', '172', '70', '350'],
 ['13', 'Minna Maarit Aalto', '2', '30', '159', '55.5', '350'],
 ['14', 'Pirjo Hannele Aalto Mattila ', '2', '32', '171', '65', '350'],
 ['15', 'Arvo Ossian Aaltonen', '1', '22', '0', '0', '35

## Importancia de countAprox

Debido a la cantidad de datos no siempre es recomendable hacer operaciones tipo count.

Por lo que 'countAprox' es la solución mas viable cuando solo queremos darnos una idea de cuántos datos podemos leer durante un tiempo determinado. Nota: el parámetro está en milisegundos

In [50]:
deportistaOlimpicoRDD.map(lambda x :[x[-1], x[:-1]]).join(
    equiposOlimpicosRDD.map(lambda x : [x[0],x[2]])
    ).count()

135427

In [51]:
deportistaOlimpicoRDD.map(lambda x :[x[-1], x[:-1]]).join(
    equiposOlimpicosRDD.map(lambda x : [x[0],x[2]])
    ).countApprox(20)

135427

# Acciones de modificacion

### Obtenemos el equipo y el deportista

Guardamos el RDD resultante en equipoDeportista

In [ ]:
equipoDeportista = deportistaOlimpicoRDD.map(lambda x :[x[-1], x[:-1]]).join(equiposOlimpicosRDD \
                                            .map(lambda x : [x[0],x[2]]))

In [ ]:
equipoDeportista.take(1)


In [ ]:
equipoDeportista.map(lambda x : (x[1][0][0],x[1][0][1:],x[1][1]) )

### Carga de resultados

Esta tabla posee las medallas que los jugadores han ganado

In [ ]:
resultado = sc.textFile("/content/drive/MyDrive/Colab Notebooks/RED.ES/Datos Ejercicios/M6/resultados.csv").map(lambda line : line.split(","))

Eliminamos todas las medallas no ganadoras y nos quedamos con el valor de la medalla y  del deportista_id

In [ ]:
resultadoGanador = resultado.filter(lambda l : 'NA' not in l[1])
resultadoGanador = resultadoGanador.map(lambda l : [l[2],l[1]])

In [ ]:
resultadoGanador.take(15)

### Obtenemos la relación buscada: deportista,pais y medalla.

In [ ]:
jugadoresMedalla =  equipoDeportista.join(resultadoGanador)
jugadoresMedalla.take(1)

[('514', ((['362', 'Isao Ko Abe', '1', '24', '177', '75'], 'JPN'), 'Silver'))]

### Agrupamos las medallas respecto a la sigla del pais jugador

In [ ]:
d = {'Gold':7, 'Silver':5, 'Bronze':4}
paisesMedallas = jugadoresMedalla.map(lambda x : (x[1][0][-1],d[x[1][1]]) )
paisesMedallas.takeSample(False,10)

### Obtenemos los valores de los puntuajes históricos de los paises jugadores

In [ ]:
from operator import add
conclusion = paisesMedallas.reduceByKey((add)).sortBy(lambda x : x[1],ascending=False)
conclusion.take(10)

[('ITA', 38559),
 ('NED', 29508),
 ('GER', 11130),
 ('JPN', 7880),
 ('NZL', 4646),
 ('TCH', 3880),
 ('TUR', 2315),
 ('RSA', 2175),
 ('BLR', 2156),
 ('ALG', 2024)]

# Estadística básica sobre los RDDs

Spark posee una suite integrada de forma natural para poder obtener estadísticas básicas.

In [ ]:
conclusion.map(lambda l : l[1]).stats()

In [ ]:
conclusion.map(lambda l : l [1]).mean()

In [ ]:
conclusion.map(lambda l : l [1]).sum()

In [ ]:
conclusion.map(lambda l : l [1]).histogram(10)

## Ejercicios Programación funcional

### Ejercicio 5
**Escribir una función que aplique el IVA a un precio.**
* Nombre de la función: apply_iva
* Argumentos: precio, iva
* Resultado: precio con iva aplicado


* Prueba a llamar a la función


In [ ]:
#Inserta aquí tu código


### Ejercicio 6
** Escribe una función que aplica la función cuadrado() a todos los elementos de una lista.**

    Parámetros:
        funcion: Recibe la función a aplicar.
        lista: Es una lista con valores que se pasarán como argumentos a funcion.
    Devuelve:
        Una lista con el resultado de aplicar la función a los valores de la lista.

In [ ]:
#Inserta aquí tu código


In [ ]:
def cuadrado(n):
    return n * n

print(aplica_funcion_lista(cuadrado, [1, 2, 3, 4]))

### Ejercicio 7
**Detectar y corregir los errores del siguiente programa que aplica el iva a una factura:**


In [ ]:
base = input('Introduce la base imponible de la factura: ')
print(aplica_iva(base, iva))

def aplica_iva(base, iva = 21):
    base = base * iva
    return base